![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None) 
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


**Preprocessed the data**:
Replaced column 13 a.k.a target variable's  values '+' and '-' with 1 and 0, respectively. 
Replaced missing values('?') and suspected mistakes (i.e. type during input) in various columns.
Fianlly, converted non-categorical columns to numerical data types and categorical columns into encoded features. 

In [12]:
cc_apps[13] = cc_apps[13].replace({'+': 1, '-': 0})
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,1


In [13]:
cc_apps[0] = cc_apps[0].replace('?','b')
cc_apps[1] = cc_apps[1].replace('?',36)
cc_apps[1] = cc_apps[1].astype(float)
cc_apps[3] = cc_apps[3].replace(['?','l'],'u')
cc_apps[4] = cc_apps[4].replace(['gg','?'],'g')
cc_apps[5] = cc_apps[5].replace('?','w')
cc_apps[6] = cc_apps[6].replace('?','v')


In [14]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    float64
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 75.6+ KB


In [15]:
X = cc_apps.drop(columns=13)
y = cc_apps[13]

In [16]:
X_encoded = pd.get_dummies(X)
# Display the encoded DataFrame
print(X_encoded.head())

       1      2     7  10   12  0_a  0_b  ...  8_f  8_t  9_f  9_t  11_g  11_p  11_s
0  30.83  0.000  1.25   1    0    0    1  ...    0    1    0    1     1     0     0
1  58.67  4.460  3.04   6  560    1    0  ...    0    1    0    1     1     0     0
2  24.50  0.500  1.50   0  824    1    0  ...    0    1    1    0     1     0     0
3  27.83  1.540  3.75   5    3    0    1  ...    0    1    0    1     1     0     0
4  20.17  5.625  1.71   0    0    0    1  ...    0    1    1    0     0     0     1

[5 rows x 41 columns]


In [17]:
#Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

In [18]:
#Converted column names to string data type
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [19]:
#Standardized the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_tesT_scaled = scaler.transform(X_test)

In [20]:
#Build and tuned a Random Forest Classifier using GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Parameter grid 
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]}

rf_classifier = RandomForestClassifier()

#GridSearchCV for Random Forest Classifier
grid_search_rf = GridSearchCV(estimator=rf_classifier, param_grid=param_grid_rf, cv=5, scoring='accuracy')

grid_search_rf.fit(X_train_scaled, y_train)

#Best parameters
best_params_rf = grid_search_rf.best_params_
print("Best Parameters for Random Forest Classifier:", best_params_rf)

#Best estimator
best_estimator_rf = grid_search_rf.best_estimator_
y_pred_rf = best_estimator_rf.predict(X_tesT_scaled)

#Calculate the accuracy score
best_score = accuracy_score(y_test, y_pred_rf)
print("Accuracy score for Random Forest Classifier:", best_score)

Best Parameters for Random Forest Classifier: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy score for Random Forest Classifier: 0.861271676300578
